In [6]:
import openslide

# Load the .ndpi image
slide = openslide.OpenSlide(r"D:\mlpr data\PKG - UPENN-GBM_v2\data\7316UP-8.ndpi")

# Check image properties
print(slide.dimensions)  # (width, height)
print(slide.level_count)  # number of resolution levels
print(slide.level_dimensions)  # resolution dimensions for each level

# Extract a region from the slide (e.g., top-left corner)
# Read a smaller region (e.g., 1000x1000 pixels) from the top-left corner
region = slide.read_region((0, 0), 0, (4000, 4000))
region.show()


(145920, 103680)
9
((145920, 103680), (72960, 51840), (36480, 25920), (18240, 12960), (9120, 6480), (4560, 3240), (2280, 1620), (1140, 810), (570, 405))


NPDI to jpg

In [10]:
import openslide
from PIL import Image
import numpy as np

# Function to downsample, normalize, and save the image
def process_and_save_ndpi(ndpi_path, output_path, tile_size=(256, 256), level=3):
    # Open the .ndpi slide
    slide = openslide.OpenSlide(ndpi_path)
    
    # Get the dimensions of the whole slide at the selected level
    slide_width, slide_height = slide.level_dimensions[level]
    
    # Create an empty image to combine all tiles (or save them one by one)
    total_image = Image.new('RGB', (slide_width, slide_height))

    # Iterate over the entire slide in tiles
    for x in range(0, slide_width, tile_size[0]):
        for y in range(0, slide_height, tile_size[1]):
            # Read a small region (tile) from the slide
            region = slide.read_region((x, y), level, tile_size)  # Read at the specified resolution level

            # Normalize the image (convert to [0, 1] range)
            region_array = np.array(region) / 255.0  # Normalize to [0, 1] if values are in [0, 255]

            # Convert the numpy array back to a PIL Image (after normalization)
            normalized_image = Image.fromarray((region_array * 255).astype(np.uint8))

            # Paste the tile into the final image (total_image)
            total_image.paste(normalized_image, (x, y))
    
    # Save the entire downsampled and normalized image
    total_image.save(output_path)
    print(f"Full image saved to {output_path}")

# Path to the NDPI file
ndpi_path = r"D:\mlpr data\PKG - UPENN-GBM_v2\data\7316UP-8.ndpi"

# Path where you want to save the processed image (e.g., output_path)
output_path = r"D:\mlpr data\processed_full_image.png"  # You can change the format as needed (PNG, JPEG)

# Call the function to process and save the entire image
process_and_save_ndpi(ndpi_path, output_path)


Full image saved to D:\mlpr data\processed_full_image.png
